# Introduction to CST-Python

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-F9AB00?style=for-the-badge&logo=googlecolab&color=525252)](https://colab.research.google.com/github/H-IAAC/CST-Python/blob/main/examples/Introduction%20to%20CST-Python.ipynb) [![Open in Github](https://img.shields.io/badge/Open%20in%20Github-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/H-IAAC/CST-Python/blob/main/examples/Introduction%20to%20CST-Python.ipynb)

The CST (Cognitive Systems Toolkit) is a code toolkit for creating agents that implements Cognitive Architectures, that is, computational models of cognitive process in the mind of living beings. The core toolkit is the [Java CST](https://cst.fee.unicamp.br/), and CST-Python is a compatible implementation in Python.

For building architectures, the CST defines three basic elements: Memory, Codelet and Mind, that will be presented in this tutorial.

Lets start by importing the CST-Python and other required modules:

In [ ]:
try:
    import cst_python
except:
    !python3 -m pip install cst_python

In [1]:
import time

import cst_python as cst

## Memories: storing data

The first element is the `Memory`. Memories are used to store data that are processed by the agent. That are many classes that implements the basic `Memory` class, but the most simple and used is the `MemoryObject`.

Lets create one, and set it's name:

In [2]:
memory = cst.MemoryObject()

memory.set_name("My Memory")

We can check that the MemoryObject is a type of Memory:

In [3]:
isinstance(memory, cst.core.entities.Memory)

True

Each Memory has a integer id, a unique identifier (that are some details about when the 'id' is really unique), and a name:

In [4]:
memory.get_id(), memory.get_name()

(0, 'My Memory')

And a `info`, the information that memory is actually storing. Because we didn't set any info, the current is `None`:

In [5]:
memory.get_info() is None

True

Let's set a info. They can be any variable of any type:

In [6]:
memory.set_info("My Memory's data")

-1

We can now see that the stored info changed. Also, each Memory has a `timestamp`, they store the time when the memory's info have changed:

In [7]:
memory.get_info(), memory.get_timestamp()

("My Memory's data", 1729269027698)

## Codelets: cognitive processes

But, a agent can't do nothing with only data. `Codelets` are elements of the architecture that process data, executing the agent's cognitive processes.

Each Codelet can have any number of inputs and outputs memories. They can also be local or global, we will only use local inputs/outputs in this example.

For creating a codelet, we need to define the methods:
- `access_memory_objects`: gets all the memories the codelet is going to use, localizable by name.
- `calculate_activation`: computes the codelet's activation. We are not going to use this now.
- `proc`: the actual function that the codelet performs, reading from the inputs, processing and setting the outputs.

It is important to note that the codelet should only get inputs in the `access_memory_objects`, not in `proc`. The content of the memories (info) can be accessed everywhere.

Also, the Codelet should be stateless. Any data necessary for its operation should be in the inputs.

Our first codelet is going to read a input value, adding one and sending this value to other memory:

In [8]:
class MyFirstCodelet(cst.Codelet):

    def __init__(self):
        super().__init__()

        self._input_mo : cst.MemoryObject | None = None
        self._output_mo : cst.MemoryObject | None = None


    def access_memory_objects(self):
        self._input_mo = self.get_input(name="InputMemory")
        self._output_mo = self.get_output(name="OutputMemory")

    def calculate_activation(self):
        pass

    def proc(self):
        read_value : float = self._input_mo.get_info()
        
        output_value = read_value + 1

        self._output_mo.set_info(output_value)

my_first_codelet = MyFirstCodelet()

Before we can use the codelet, we need to create and add it's input and output, and execute `access_memory_objects`:

In [9]:
input_memory = cst.MemoryObject()
input_memory.set_name("InputMemory")

output_memory = cst.MemoryObject()
output_memory.set_name("OutputMemory")

my_first_codelet.add_input(input_memory)
my_first_codelet.add_output(output_memory)

my_first_codelet.access_memory_objects()

Lets test it by setting the info, running the `proc` and checking if the output value is correct:

In [10]:
input_memory.set_info(0)

my_first_codelet.proc()

output_memory.get_info()

1

## Mind: organizing everything

Now we can create all the data and cognitive process of the agent, but they need to be manually managed.

The `Mind` element contains all the memories in its `RawMemory` and all the codelets in its `Coderack`. It also manage the execution of all the codelets and  is the expected way to create an agent with CST.

Lets create a Mind:

In [11]:
mind = cst.Mind()

Using a Mind, all the MemoryObjects need to be create by it (now each memory is guaranteed to have a unique id). We can also pass a default value, in this case `0` to the "InputMemory":

In [12]:
input_memory = mind.create_memory_object("InputMemory", 0)
output_memory = mind.create_memory_object("OutputMemory")

We create the codelet and add it's inputs and outputs as before:

In [13]:
my_first_codelet = MyFirstCodelet()
my_first_codelet.add_input(input_memory)
my_first_codelet.add_output(output_memory)

But now insert it into the Mind:

In [14]:
mind.insert_codelet(my_first_codelet)

When running using a Mind, it will run each codelet in a separated thread at a fixed rate. The default `time step` is 300 ms. Lets change it to 100 ms for a faster execution:

In [15]:
my_first_codelet.time_step = 100

The `start` method start the Mind and the execution of all the codelets:

In [16]:
mind.start()

After waiting our codelet run by 110 ms, we can check the added value in the ouput memory: 

In [17]:
time.sleep(0.110)

output_memory.get_info()

1

We can also change the input memory, wait and check the modified value in the output:

In [18]:
input_memory.set_info(123)

time.sleep(0.110)

output_memory.get_info()

124

The `shutdown` method stops the execution of the Mind:

In [19]:
mind.shutdown()